In [1]:
import boto3
import subprocess
import getpass

# ---------- PARAMETERS ----------
region = boto3.Session().region_name
account_id = boto3.client('sts').get_caller_identity()['Account']
ecr_repo_name = "financial-fraud-training"
ecr_image_tag = "1.0.1"
# ecr_image_tag = "2.0.0"
ngc_image = f"nvcr.io/nvidia/cugraph/{ecr_repo_name}:{ecr_image_tag}"

In [2]:
# --- 1. Login to NGC (requires NGC API key) ---
# Replace <YOUR_NGC_API_KEY> with your real NGC key or load from Secrets Manager
NGC_API_KEY = "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"
ngc_login_cmd = f"docker login nvcr.io --username '$oauthtoken' --password {NGC_API_KEY}"
subprocess.run(ngc_login_cmd, shell=True, check=True)
print("✅ Logged in to nvcr.io")

WARNING! Using --password via the CLI is insecure. Use --password-stdin.


Login Succeeded
✅ Logged in to nvcr.io


WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [3]:
# --- 2. Pull the NGC container ---
subprocess.run(f"docker pull {ngc_image}", shell=True, check=True)
print("✅ Pulled container from NGC")

1.0.1: Pulling from nvidia/cugraph/financial-fraud-training
de44b265507a: Pulling fs layer
75f58769314d: Pulling fs layer
4f4fb700ef54: Pulling fs layer
5a9563dd6b7e: Pulling fs layer
e5a680752b35: Pulling fs layer
4f1559b21c0f: Pulling fs layer
8f3004ab24c5: Pulling fs layer
9c281553aeb1: Pulling fs layer
0e3fff093155: Pulling fs layer
4eb34e451322: Pulling fs layer
07e412f82f03: Pulling fs layer
2dabfd5dddb1: Pulling fs layer
c50f8ea39070: Pulling fs layer
d8653b71b07b: Pulling fs layer
f7d0dd940fc6: Pulling fs layer
25544f040cbe: Pulling fs layer
e4ac16b4865e: Pulling fs layer
bfde45aa1d7e: Pulling fs layer
2da96e8d8318: Pulling fs layer
d20c024db52a: Pulling fs layer
664c6a22ad93: Pulling fs layer
4b98a953a32b: Pulling fs layer
2dfd77b797d2: Pulling fs layer
10952d5b1cc1: Pulling fs layer
155223c296bb: Pulling fs layer
9577583d0756: Pulling fs layer
3f198363f368: Pulling fs layer
733c3134baa1: Pulling fs layer
d554d4d2d207: Pulling fs layer
e4ac16b4865e: Waiting
bfde45aa1d7e: Waiti

In [4]:
# --- 3. Create ECR repository if not exists ---
ecr_client = boto3.client('ecr', region_name=region)
try:
    ecr_client.create_repository(repositoryName=ecr_repo_name)
    print("✅ Created new ECR repository")
except ecr_client.exceptions.RepositoryAlreadyExistsException:
    print("ℹ️ ECR repository already exists")


ℹ️ ECR repository already exists


In [5]:
# --- 4. Authenticate Docker with ECR ---
login_password = subprocess.check_output(
    f"aws ecr get-login-password --region {region}", shell=True
).decode('utf-8').strip()

In [6]:
subprocess.run(
    f"docker login --username AWS --password {login_password} "
    f"{account_id}.dkr.ecr.{region}.amazonaws.com",
    shell=True, check=True
)
print("✅ Logged in to ECR")


Login Succeeded
✅ Logged in to ECR


WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [7]:
# --- 5. Tag & Push image ---
subprocess.run(f"docker tag {ngc_image} {account_id}.dkr.ecr.us-east-1.amazonaws.com/{ecr_repo_name}:{ecr_image_tag}", shell=True, check=True)
print(f"✅ Tagged image as {ngc_image} {account_id}.dkr.ecr.us-east-1.amazonaws.com/{ecr_repo_name}:{ecr_image_tag}")

✅ Tagged image as nvcr.io/nvidia/cugraph/financial-fraud-training:1.0.1 176843580427.dkr.ecr.us-east-1.amazonaws.com/financial-fraud-training:1.0.1


In [10]:
!docker images

REPOSITORY                                                              TAG       IMAGE ID       CREATED       SIZE
176843580427.dkr.ecr.us-east-1.amazonaws.com/financial-fraud-training   1.0.1     27d5d51b88e7   5 weeks ago   33.6GB
nvcr.io/nvidia/cugraph/financial-fraud-training                         1.0.1     27d5d51b88e7   5 weeks ago   33.6GB


In [12]:
subprocess.run(f"docker push {account_id}.dkr.ecr.us-east-1.amazonaws.com/{ecr_repo_name}:{ecr_image_tag}", shell=True, check=True)
print(f"✅ Successfully pushed {ngc_image} to ECR: {account_id}.dkr.ecr.us-east-1.amazonaws.com/{ecr_repo_name}:{ecr_image_tag}")

The push refers to repository [176843580427.dkr.ecr.us-east-1.amazonaws.com/financial-fraud-training]
00a3521476e1: Preparing
9069331183ed: Preparing
0b0b8c74460d: Preparing
116ae6f7a531: Preparing
e6cb6fddf21b: Preparing
c7fdea842d22: Preparing
98adafc13fb8: Preparing
bcc30b74dcdb: Preparing
53f028f4a002: Preparing
111746b6db13: Preparing
5faf93c7821d: Preparing
ace8fb5c62e6: Preparing
5f70bf18a086: Preparing
7853fa41ed73: Preparing
8ff7e1bcd113: Preparing
535e620592d3: Preparing
c9bad26eecba: Preparing
1716ff5ad373: Preparing
87892cc7abdf: Preparing
5f70bf18a086: Preparing
17d4cc7eec42: Preparing
c1a1c664a6b0: Preparing
73d3693f84a1: Preparing
b535aad9783f: Preparing
546e9fe48264: Preparing
f85c548d7388: Preparing
fc5fd61d6e22: Preparing
635862118e8f: Preparing
66b8de556eb7: Preparing
3b6b2df5fa9f: Preparing
d25b7414c1a4: Preparing
a83622292fda: Preparing
dd4aa2779e68: Preparing
f375e671e0ac: Preparing
f84f3c952004: Preparing
2b8af5268d57: Preparing
5f70bf18a086: Preparing
fbbe166ef1

In [13]:
ecr_client.describe_repositories(repositoryNames=[ecr_repo_name])["repositories"][0]


{'repositoryArn': 'arn:aws:ecr:us-east-1:176843580427:repository/financial-fraud-training',
 'registryId': '176843580427',
 'repositoryName': 'financial-fraud-training',
 'repositoryUri': '176843580427.dkr.ecr.us-east-1.amazonaws.com/financial-fraud-training',
 'createdAt': datetime.datetime(2025, 10, 5, 13, 28, 52, 749000, tzinfo=tzlocal()),
 'imageTagMutability': 'MUTABLE',
 'imageScanningConfiguration': {'scanOnPush': False},
 'encryptionConfiguration': {'encryptionType': 'AES256'}}

In [62]:
!docker run -i {ngc_image} -e NIM_DISABLE_MODEL_DOWNLOAD=True -e NGC_API_KEY={NGC_API_KEY} 

In [8]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'ls /opt/nim/lib/financial_fraud_training'

Dockerfile
README.md
__init__.py
docs
example_training_config_graphsage_xgboost.json
example_training_config_xgboost.json
main.py
src
tests
utils


In [17]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'pip uninstall -y pyg-lib && pip install --no-cache-dir pyg-lib -f https://data.pyg.org/whl/torch-2.6.0+cu124.html && python -c "import pyg_lib; print(pyg_lib.__file__); print(pyg_lib.__version__)"'

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/faiss-1.7.4-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/pyg_lib-0.4.0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
Found existing installation: pyg_lib 0.4.0
Uninstalling pyg_lib-0.4.0:
  Successfully uninstalled pyg_lib-0.4.0
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/faiss-1.7.4-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at

In [16]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'python -c "import pyg_lib; print(pyg_lib.__file__)"'

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.12/dist-packages/pyg_lib-0.4.0-py3.12-linux-x86_64.egg/pyg_lib/__init__.py", line 34, in <module>
    load_library('libpyg')
  File "/usr/local/lib/python3.12/dist-packages/pyg_lib-0.4.0-py3.12-linux-x86_64.egg/pyg_lib/__init__.py", line 31, in load_library
    torch.ops.load_library(spec.origin)
  File "/usr/local/lib/python3.12/dist-packages/torch/_ops.py", line 1357, in load_library
    ctypes.CDLL(path)
  File "/usr/lib/python3.12/ctypes/__init__.py", line 379, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /usr/local/lib/python3.12/dist-packages/pyg_lib-0.4.0-py3.12-linux-x86_64.egg/libpyg.so: undefined symbol: _ZNK5torch8autograd4Node4nameB5cxx11Ev


In [12]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'python -c "import torch; print(torch.__file__); print(torch.__version__)"'

/usr/local/lib/python3.12/dist-packages/torch/__init__.py
2.6.0+cu124


In [11]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'cat /opt/nim/lib/financial_fraud_training/main.py'

# Copyright (c) 2025, NVIDIA CORPORATION. All rights reserved.

# This source code and/or documentation ("Licensed Deliverables") are
# subject to NVIDIA intellectual property rights under U.S. and
# international Copyright laws.

import logging
import argparse

from src.validate_and_launch import validate_config_and_run_training


def setup_logging(level=logging.INFO):
    """Configure logging format and level."""
    logging.basicConfig(
        level=level,
        format="%(asctime)s [%(levelname)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )


setup_logging()

if __name__ == "__main__":

    usage_msg = (
        'docker run --cap-add SYS_NICE -it --rm --gpus "device = 0" '
        "-v <path_to_data_dir_on_host>:/data "
        "-v <path_to_model_output_dir_on_host>:/trained_models "
        "financial-fraud-training --config <path_to_training_config_on_host.json>"
    )
    parser = argparse.ArgumentParser(
        description="This program requires a training config

In [69]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'ls /opt/nim/'

LICENSE
VERSION
data_models.py
etc
inference.py
lib
start_server.sh


In [12]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'cat /opt/nim/inference.py'


# SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
# SPDX-License-Identifier: LicenseRef-NvidiaProprietary
#
# NVIDIA CORPORATION, its affiliates and licensors retain all intellectual
# property and proprietary rights in and to this material, related
# documentation and any modifications thereto. Any use, reproduction,
# disclosure or distribution of this material and related documentation
# without an express license agreement from NVIDIA CORPORATION or
# its affiliates is strictly prohibited.

from __future__ import annotations

import json
import logging
import os
import sys

from contextlib import asynccontextmanager
from typing import Any


from data_models import ExampleRequest, ExampleResponse

from fastapi import FastAPI, HTTPException

from nimlib.nim_inference_api_builder.http_api import HttpNIMApiInterface

from pydantic import TypeAdapter, ValidationError
from lib.financial_fraud_training.src.config_schema import FullConfig, 

In [68]:
!docker run --entrypoint "/bin/bash"  {ngc_image} -c 'cat /opt/nim/start_server.sh'

    #!/usr/bin/env bash
    set -eu

    # SPDX-FileCopyrightText: Copyright (c) 2024 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
    # SPDX-License-Identifier: LicenseRef-NvidiaProprietary

    
    start_server


In [54]:
!docker inspect {ngc_image} 

docker inspect nvcr.io/nvidia/cugraph/financial-fraud-training:1.0.1
[
    {
        "Id": "sha256:27d5d51b88e7fecc5d63d5ac325d844715b03565b82f849cedc55b258e026ab7",
        "RepoTags": [
            "176843580427.dkr.ecr.us-east-1.amazonaws.com/financial-fraud-training:1.0.1",
            "nvcr.io/nvidia/cugraph/financial-fraud-training:1.0.1"
        ],
        "RepoDigests": [
            "176843580427.dkr.ecr.us-east-1.amazonaws.com/financial-fraud-training@sha256:305facef8f647bd3986c2aebfab1343d35c778f1758adb4df23c3466f9dfade8",
            "nvcr.io/nvidia/cugraph/financial-fraud-training@sha256:3e7e68657393df74e84811017562bb75ad69871335c24e0e38ff031b3cf0da3e"
        ],
        "Parent": "",
        "Comment": "buildkit.dockerfile.v0",
        "Created": "2025-09-02T12:52:03.405102455+02:00",
        "Container": "",
        "ContainerConfig": {
            "Hostname": "",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachS